In [26]:
import requests
import json
import math
import time
import os
import uuid
import platform
import logging
import subprocess
from subprocess import Popen, PIPE
import pandas as pd
from dateutil.relativedelta import relativedelta
import datetime
from datetime import timedelta
from datetime import datetime
from dateutil import tz
import dateutil.parser as dp
import pandas.io.gbq as gbq
from google.cloud import bigquery
from google.cloud import storage

In [3]:
app_path = os.getcwd()
gspath = 'gs://sarasmaster'
os.chdir(os.getcwd())
filesep = '\\' if platform.system() == 'Windows' else '/'
gssep = '/'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "creds" + filesep + "sarasmaster-524142bf5547.json"
gcspath = 'C:\\Users\\kabhi\\AppData\\Local\\Google\\Cloud SDK\\google-cloud-sdk\\bin'
os.environ["PATH"] += os.pathsep + gcspath
batfile = app_path + filesep + 'movetogcs.bat' if platform.system() == 'Windows' else app_path + filesep + 'movetogcs.sh'
delimitertype = 'CSV'
loadtype = 'WRITE_APPEND'
skipheader = 1

In [ ]:
def loadlocalfiletogooglestorage(batfile, source_file_name, dest_file_name):
    pass_arg=[]
    pass_arg.append(batfile)
    pass_arg.append(source_file_name)
    pass_arg.append(dest_file_name)
    p = Popen(pass_arg, stdout=PIPE, stderr=PIPE)
    output, errors = p.communicate()
    p.wait() # wait for process to terminate
    print(output)
    print(errors)

In [ ]:
def loadfiletobigquery(file_name, dataset_id, table_name, delimitertype, loadtype, skipheader):
    client = bigquery.Client()
    table_ref = client.dataset(dataset_id).table(table_name)
    job_config = bigquery.LoadJobConfig()
    if skipheader is not None:
        job_config.skip_leading_rows = skipheader
    job_config.source_format = delimitertype
    #if delimitertype == bigquery.SourceFormat.CSV:
        #job_config.autodetect = True
    job_config.write_disposition = loadtype

    load_job = client.load_table_from_uri(
        file_name,
        table_ref,
        job_config=job_config)  # API request

    assert load_job.job_type == 'load'

    load_job.result()  # Waits for table load to complete.

    assert load_job.state == 'DONE'

In [53]:
def addcolumnames(dfdata):
    dfdata.columns = ['dummy','vendor_id','vendor_name','item_number','item_description','div_id','dept_id','class_id','sub_class_id',
    'item_rank','store_count','upc_no','vendor_part_no','brand_id','brand_name','purch_cost_avg','retail_price_cur','feature_code',
    'season_code','total_sales_ty_units','total_sales_ly_units','total_sales_units_chg_per','total_sales_ty','total_sales_ly','total_sales_chg_per',
    'comp_sales_ty_units','comp_sales_ly_units','comp_sales_units_chg_per','comp_sales_ty','comp_sales_ly','comp_sales_chg_per',
    'noncomp_sales_ty_units','noncomp_sales_ly_units','noncomp_sales_units_chg_per','noncomp_sales_ty','noncomp_sales_ly','noncomp_sales_chg_per',
    'dotcom_sales_ty_units','dotcom_sales_ly_units','dotcom_sales_units_chg_per','dotcom_sales_ty','dotcom_sales_ly','dotcom_sales_chg_per',
    'totaleoh_sales_ty_units','totaleoh_sales_ly_units','totaleoh_sales_units_chg_per','totaleoh_sales_ty','totaleoh_sales_ly','totaleoh_sales_chg_per',
    'totalwos','dceoh_sales_ty_units','dceoh_sales_ly_units','dceoh_sales_units_chg_per','dceoh_sales_ty','dceoh_sales_ly','dceoh_sales_chg_per',
    'dc_on_order_units','dc_dotcom_order_units','dc_dotcom_wos','dc_total_wos','storeeoh_sales_ty_units','storeeoh_sales_ly_units','storeeoh_sales_units_chg_per',
    'storeeoh_sales_ty','storeeoh_sales_ly','storeeoh_sales_chg_per','storeeoh_comp_units','storeeoh_noncomp_units','store_on_order_units',
    'store_in_transit_units','store_avg_sales','store_comp_wos','store_noncomp_wos','store_total_wos']
    return dfdata

In [67]:
def readfiles(filename):
    df = pd.read_excel(filename, sheet_name=0,header=None)
    dfdata = df[9:]
    dfdata = dfdata.reset_index(drop=True)
    dfheaders = df[:9]
    date_str = dfheaders.iloc[2][3].split(':')[1]
    end_date_str = date_str.split('-')[1]
    end_date_str = end_date_str.strip()
    start_date_str = date_str.split('-')[0]
    start_date_str = start_date_str.strip()
    start_date = datetime.strptime(start_date_str, '%m/%d/%Y')
    end_date = datetime.strptime(end_date_str, '%m/%d/%Y')
    dfdata = addcolumnames(dfdata)
    no_week = start_date.isocalendar()[1]
    currency_cd = 'USD'
    quarter = pd.Timestamp(start_date).quarter
    year = pd.Timestamp(start_date).year
    load_id = uuid.uuid4()
    dfdata['load_id'] = load_id
    dfdata['start_date'] = start_date
    dfdata['end_date'] = end_date
    dfdata['year'] = year
    dfdata['qtr'] = quarter
    dfdata['week'] = no_week
    dfdata['currency_code'] = currency_cd
    print(start_date)
    print(end_date)   
    dfdata = dfdata.drop('dummy', axis=1)
    localfilename = app_path + filesep + 'kopari' + filesep + 'ulta' + filesep + 'ulta_sales_inv_'
    gsfilepath = gspath + gssep + 'kopari' + gssep + 'ulta'
    localfilename = localfilename + start_date.strftime('%Y%m%d') + '_' + end_date.strftime('%Y%m%d') + '.csv'
    dfdata.to_csv(localfilename, index=False)
    #loadlocalfiletogooglestorage(batfile, localfilename, gsfilepath)

In [68]:
rootdir = app_path + filesep + 'kopari' + filesep + 'ulta'
dfdata = pd.DataFrame()
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        filepath = subdir + os.sep + file
        if filepath.endswith(".xls") or filepath.endswith(".xlsx"):
            print(file)
            readfiles(filepath)

Sales and Inventory Report - 2018 FW02 - KOCO LIFE (49509).xlsx
2018-02-11 00:00:00
2018-02-17 00:00:00
Sales and Inventory Report - 2018 FW03 - KOCO LIFE (49509).xlsx
2018-02-18 00:00:00
2018-02-24 00:00:00
Sales and Inventory Report - 2018 FW06 - KOCO LIFE (49509).xlsx
2018-03-11 00:00:00
2018-03-17 00:00:00
Sales and Inventory Report - 2018 FW07 - KOCO LIFE (49509).xlsx
2018-03-18 00:00:00
2018-03-24 00:00:00
Sales and Inventory Report - 2018 FW08 - KOCO LIFE (49509).xlsx
2018-03-25 00:00:00
2018-03-31 00:00:00
Sales and Inventory Report - 2018 FW09 - KOCO LIFE (49509).xlsx
2018-04-01 00:00:00
2018-04-07 00:00:00
Sales and Inventory Report - 2018 FW11 - Week ending 4_22_18.xlsx
2018-04-15 00:00:00
2018-04-21 00:00:00
Sales and Inventory Report - 2018 FW12 - KOCO LIFE (49509).xlsx
2018-04-22 00:00:00
2018-04-28 00:00:00
Sales and Inventory Report 4_14_18.xlsx
2018-04-08 00:00:00
2018-04-14 00:00:00
Ulta Sales and Inventory Report - 2018 FW04 - KOCO LIFE (49509).xlsx
2018-02-25 00:00:

In [63]:
df = pd.read_excel('C:\\Users\\kabhi\\Desktop\\datamaster\\kopari\\ulta\\Sales and Inventory Report - 2018 FW02 - KOCO LIFE (49509).xlsx', sheet_name=0,header=None)
dfdata = df[9:]
dfdata = dfdata.reset_index(drop=True)
dfheaders = df[:9]


In [64]:
dfdata.head()

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,NaN,49509,KOCO LIFE,2523646,KOPARI COCONUT ROSE TONER 5.10OZ,1,14,115,2168,F,...,NaN,NaN,NaN,NaN,3430,NaN,8,NaN,NaN,428.75
1,NaN,49509,KOCO LIFE,2523648,KOPARI COCONUT FACE CREAM 2.5OZ,1,14,115,1168,F,...,NaN,NaN,NaN,NaN,3477,NaN,7,NaN,NaN,496.714
2,NaN,49509,KOCO LIFE,2523653,KOPARI COCONUT CLEANSING OIL 5.1OZ,1,14,115,1167,F,...,NaN,NaN,NaN,NaN,3512,NaN,10,NaN,NaN,351.2
3,NaN,49509,KOCO LIFE,2523654,KOPARI COCONUT MELT 5.1OZ,1,14,116,1171,F,...,NaN,NaN,NaN,NaN,4665,NaN,31,NaN,NaN,150.484


In [65]:
dfdata = addcolumnames(dfdata)

In [66]:
dfdata.drop('dummy', axis=1)


,vendor_id,vendor_name,item_number,item_description,div_id,dept_id,class_id,sub_class_id,item_rank,store_count,...,storeeoh_sales_ly,storeeoh_sales_chg_per,storeeoh_comp_units,storeeoh_noncomp_units,store_on_order_units,store_in_transit_units,store_avg_sales,store_comp_wos,store_noncomp_wos,store_total_wos
0,49509,KOCO LIFE,2523646,KOPARI COCONUT ROSE TONER 5.10OZ,1,14,115,2168,F,NaN,...,NaN,NaN,NaN,NaN,3430,NaN,8,NaN,NaN,428.75
1,49509,KOCO LIFE,2523648,KOPARI COCONUT FACE CREAM 2.5OZ,1,14,115,1168,F,NaN,...,NaN,NaN,NaN,NaN,3477,NaN,7,NaN,NaN,496.714
2,49509,KOCO LIFE,2523653,KOPARI COCONUT CLEANSING OIL 5.1OZ,1,14,115,1167,F,NaN,...,NaN,NaN,NaN,NaN,3512,NaN,10,NaN,NaN,351.2
3,49509,KOCO LIFE,2523654,KOPARI COCONUT MELT 5.1OZ,1,14,116,1171,F,NaN,...,NaN,NaN,NaN,NaN,4665,NaN,31,NaN,NaN,150.484
